# **Using HST COS data for Lyα** (Sujay)

In [ ]:
import altair as alt
import corv
import numpy as np
import pandas as pd

from astropy.table import Table
from pathlib import Path
from scripts import make_bins
from tqdm import tqdm
from typing import Tuple

In [25]:
cos_objs = pd.read_csv('data/hst_cos/STAR_WD_COS_new.info', sep=r'\s,\s+', engine='python', usecols=[1, 2, 3], names=['ID', 'FLAG', 'META'])
cos_objs = cos_objs[cos_objs.META.str.contains('DA')]
cos_objs = cos_objs[~cos_objs.META.str.contains('PULSAR')].ID
cos_objs

7      TYC-6917-654-1
8      TYC-7218-934-1
9      TYC-9151-303-1
13           BPM71214
16            G191B2B
            ...      
289        WD2126+734
290        WD2230-125
291        WD2319+691
293        WD2336-079
296        PG1508+549
Name: ID, Length: 246, dtype: object